# Assignment 1

## Part I

In [1]:
from IPython.core.debugger import set_trace

In [2]:
#CONSTANTS:

LIMIT = 100
#LIMIT = 40

### Class Vertex

In [3]:
class Vertex:
    def __init__(self, key, number_of_people):
        self.key = key
        self.number_of_people = number_of_people
    def get_key(self):
        return self.key
    def minimum(vertex1,vertex2):
        if vertex1.key <= vertex2.key:
            return vertex1
        return vertex2
    def maximum(vertex1,vertex2):
        if vertex1.key >= vertex2.key:
            return vertex1
        return vertex2
    def get_vertex(vertices, key):
        for vertex in vertices:
            if vertex.key == key:
                return vertex
        print("No vertex found")
        return None
    def is_same_vertex(self, vertex2):
        if self == vertex2: return True
        if self == None and vertex2!= None: return False
        if self != None and vertex2== None: return False
        if self.key != vertex2.key or self.number_of_people != vertex2.number_of_people:
            return False
        return True
    def to_print(self):
        return "{0}, ({1} people)".format(self.key, self.number_of_people)

### Class Edge

In [4]:
import numpy as np
class Edge:
    def __init__(self, vertex1, vertex2, weight, key):
        self.vertex1 = Vertex.minimum(vertex1, vertex2)
        self.vertex2= Vertex.maximum(vertex1, vertex2)
        self.weight = weight
        self.blocked = False
        self.key=key
    def get_key(self):
        return self.key
    def occurs_in_vertex_key(self, vertex_number):
        return self.vertex1.key==vertex_number or self.vertex2.key==vertex_number
    def occurs_in_vertices_keys(self, v1, v2):
        return self.vertex1.key == np.minimum(v1,v2) and self.vertex2.key == np.maximum(v1,v2)
    def occurs_in_vertex_element(self, vertex_element):
        return self.vertex1.key==vertex_element.key or self.vertex2.key==vertex_element.key
    def occurs_in_vertices_elements(self, v1, v2):
        return self.vertex1.key == Vertex.minimum(v1,v2).key and self.vertex2.key == Vertex.maximum(v1,v2).key
    def get_other_end(self, vertex):
        if self.vertex1.key == vertex.key:
            return self.vertex2
        if self.vertex2.key == vertex.key:
            return self.vertex1
    def is_same_edge(self, edge2):
        if not self.vertex1.is_same_vertex(edge2.vertex1): return False
        if not self.vertex2.is_same_vertex(edge2.vertex2): return False
        if self.weight != edge2.weight: return False
        if self.blocked!=edge2.blocked: return False
        if self.key!=edge2.key: return False
        return True
    def to_print(self):
        pr = "W{}: ({}, {}) weight {}".format(self.key, self.vertex1.key, self.vertex2.key,self.weight)
        if self.blocked:
            pr+=" (BLOCKED)"
        return pr

### Class Graph

In [5]:
import sys
#TODO REMOVE SYS and ERROR
LARGE_NUMBER=999999999999
class Graph:
    def __init__(self, vertices=[], edges=[]):
        self.vertices = vertices
        self.vertices.sort(key = Vertex.get_key)
        self.edges = edges
        self.edges.sort(key = Vertex.get_key)
        self.blocked_edges = []
        
    def is_same_graph(self, graph2):
        vertices2 = graph2.vertices
        edges2 = graph2.edges
        blocked_edges2 = graph2.blocked_edges
        if len(self.vertices)!=len(vertices2): return False
        if len(self.edges)!=len(edges2): return False
        if len(self.blocked_edges)!=len(blocked_edges2): return False
        for i in range(len(self.vertices)):
            if not Vertex.is_same_vertex(self.vertices[i], vertices2[i]): return False
        for i in range(len(self.edges)):
            if not Edge.is_same_edge(self.edges[i], edges2[i]): return False
        for i in range(len(self.blocked_edges)):
            if not Edge.is_same_edge(self.blocked_edges[i], blocked_edges2[i]): return False
        return True
    def input_from_text(self,file):
        pass
    def has_vertex(self, vertex):
        return vertex in self.vertices
    def are_connected(self, v1,v2):
        for edge in self.edges:
            if edge.occurs_in_vertices_keys(v1,v2):
                return True
            return False
    def to_print(self):
        pr = "Vertices:\n"
        for vertex in self.vertices:
            pr+="\t"+vertex.to_print()+"\n"
        pr+="Edges:\n"
        for e in self.edges:
            pr+="\t"+e.to_print()+"\n"
        for e in self.blocked_edges:
            pr+="\t"+e.to_print()+"\n"
        return pr
    def get_vertex(self, vertex_number):
        for vertex in self.vertices:
            if vertex.key == vertex_number:
                return vertex
        print("No vertex found.")
        return None
    def find_distance(self, vertex1, vertex2):
        for edge in self.edges:
            if edge.occurs_in_vertices_elements(vertex1,vertex2):
                return edge.weight
    def neighbours_of(self, vertex):
        neighbours = []
        for edge in self.edges:
            if edge.occurs_in_vertex_element(vertex):
                neighbours.append(edge.get_other_end(vertex))
        return neighbours
    def Dijkstra(self, start, end):
        length = len(self.vertices)
        visited = [False]*length
        dist = [float("inf")]*length
        prev = [None]*length
        dist[self.vertices.index(start)] = 0
        while not all(visited):
            v_idx = min([i for i, x in enumerate(visited) if not x], key= dist.__getitem__)
            
            v = self.vertices[v_idx]
            if v==end:
                break
            if dist[v_idx] == float("inf"): #there is no path from start to end
                return None
            visited[v_idx] = True
            neighbours = self.neighbours_of(v)
            for neighbour in neighbours:
                neig_idx = self.vertices.index(neighbour)
                if visited[neig_idx]:
                    continue
                alt = dist[v_idx] + self.find_distance(v,neighbour)
                if alt<dist[neig_idx]:
                    dist[neig_idx] = alt
                    prev[neig_idx] = v
        path = [end]
        vertex = end
        while vertex != start:
            ancestor = prev[self.vertices.index(vertex)]
            path = [ancestor]+path
            vertex = ancestor
        return path
    
    
    def get_adjacent_edges(self, vertex):
        adjacent_edges=[]
        for edge in self.edges:
            if edge.occurs_in_vertex_element(vertex):
                adjacent_edges.append(edge)
        return adjacent_edges
    def block(self, edge):
        self.edges.remove(edge)
        self.blocked_edges.append(edge)
        edge.blocked = True
        self.blocked_edges.sort(key = Vertex.get_key)
    
    def more_people(self):
        for vertex in self.vertices:
            if vertex.number_of_people > 0: return True
        return False

### Class Action

In [6]:
TRAVERSE = 1
TERMINATE = 2
NO_OP=3
BLOCK = 4

In [7]:
class Action:
    def __init__(self, action_type, argument=None): #argument is a vertx or an edge
        self.action_type=action_type
        self.argument = argument

### Class Position

In [8]:
class Position:
    def __init__(self, graph, vertex_number):
        #input("start position")
        self.graph = graph
        start_vertex = graph.get_vertex(vertex_number)
        self.start = start_vertex
        self.end = None
        self.distance_to_destination = 0
        #input("end position:")
        #input("distance_to_destination: {}".format(self.distance_to_destination))
    def is_same_position(self, position2):
        if not Vertex.is_same_vertex(self.start, position2.start): return False
        if not Vertex.is_same_vertex(self.end, position2.end): return False
        if self.distance_to_destination != position2.distance_to_destination: return False
        return True
    def in_vertex(self):
        return self.distance_to_destination == 0
    def position_move(self,graph = None, destination=None):
        if not self.in_vertex():
            self.distance_to_destination -= 1
            if self.distance_to_destination == 0:
                self.start = self.end
                self.end = None
            return
        self.end = destination
        self.distance_to_destination = graph.find_distance(self.start, self.end)
        self.distance_to_destination -= 1
        if self.distance_to_destination == 0:
            self.start = self.end
            self.end = None
    def to_print(self):
        if self.in_vertex():
            return "vertex: "+str(self.start.key)
        return "From {} to {}. Distance to destination: {}".format(self.start.key, self.end.key, self.distance_to_destination)

### Class Agent

In [9]:
class Agent:
    def __init__(self,observations, vertex_position,key):
        self.actions_counter = 0
        self.score = 0
        self.is_terminated = False
        self.graph = observations[0]
        self.position = Position(self.graph, vertex_position)
        self.in_calculation = False
        
        self.score += self.position.start.number_of_people
        self.position.start.number_of_people = 0
        self.key = key
    def terminate(self):
        self.is_terminated = True
    
    def agent_in_vertex(self):
        if self.position.in_vertex():
            self.score += self.position.start.number_of_people
            self.position.start.number_of_people = 0
    
    def agent_move(self, destination = None):
        self.actions_counter+=1
        self.agent_in_vertex()
        if not self.position.in_vertex():
            self.position.position_move()
        else:
            self.position.position_move(self.graph,destination)
        self.agent_in_vertex()
        
        
        
    def agent_terminate(self):
        self.is_terminated = True
    def agent_block(self, edge):
        self.graph.block(edge)
        self.actions_counter+=1
    
    def to_print(self):
        pr = type(self).__name__+"\n"
        pr+="actions_counter: "+str(self.actions_counter)+"\n"
        pr+="score: "+str(self.score)+"\n"
        pr+="is_terminated: "+str(self.is_terminated)+"\n"
        pr+="position:\n"
        poistion_str = self.position.to_print()
        lines = poistion_str.splitlines()
        for i, line in enumerate(lines):
            pr+="\t"+line
        pr+="\nIn a calculation: {}".format(self.in_calculation)
        return pr

In [10]:
class Human_Agent(Agent):
    def __init__(self, observations, position,key):
        super().__init__(observations, position,key)
        
    def get_action(self):
        if not self.position.in_vertex():
            return Action(TRAVERSE)
        v = input("Agent {}, what vertex do you want to go? To Terminate, input 't'.\n".format(self.key))
        if (v == 't'):
            return Action(TERMINATE)
        v=int(v)
        vertex = self.graph.get_vertex(v)
        return Action(TRAVERSE, vertex)
    def to_print(self):
        return super().to_print()

In [11]:
class Greedy_Agent(Agent):
    def __init__(self, observations, position,key):
        super().__init__(observations, position,key)
        
    def get_action(self):
        if not self.position.in_vertex():
            return Action(TRAVERSE)
        min_len = float('inf')
        for vertex in self.graph.vertices:
        #for vertex in self.graph.vertices:
        #    if Vertex.is_same_vertex(vertex, self.position.start): continue
            if vertex.number_of_people > 0:
                path = self.graph.Dijkstra(self.position.start, vertex)
                if path == None:
                    continue
                if len(path)<min_len:
                    min_path = path
                    min_len = len(path)
        if min_len == float('inf'):
            return Action(TERMINATE)
        vertex = path[1]
        return Action(TRAVERSE, vertex)
    
    def to_print(self):
        return super().to_print()

In [12]:
class Saboteur_Agent(Agent):
    def __init__(self, observations, position, num_of_NO_OPS,key):
        super().__init__(observations, position,key)
        self.num_of_NO_OPS = num_of_NO_OPS
        self.NO_OPS_remains = num_of_NO_OPS
        self.need_traverse = False
    def get_action(self):
        if not self.position.in_vertex():
            return Action(TRAVERSE)
        if self.need_traverse:
            self.need_traverse = False
            self.NO_OPS_remains = self.num_of_NO_OPS
            lightest_edge = self.get_lightest_edge()
            if lightest_edge == None:
                return Action(TERMINATE)
            vertex = lightest_edge.get_other_end(self.position.start)
            return Action(TRAVERSE, vertex)
        if self.NO_OPS_remains > 0:
            self.NO_OPS_remains-=1
            return Action(NO_OP)
        lightest_edge = self.get_lightest_edge()
        if lightest_edge == None:
            return Action(TERMINATE)
        self.need_traverse = True
        return Action(BLOCK, lightest_edge)
    
    def get_lightest_edge(self):
        adjacent_edges = self.graph.get_adjacent_edges(self.position.start)
        if not adjacent_edges:
            return None
        lightest_edge = adjacent_edges[0]
        for edge in adjacent_edges:
            if edge.weight<lightest_edge.weight or (edge.weight==lightest_edge and edge.key < lightest_edge.key):
                lightest_edge = edge
        return lightest_edge
    
    def to_print(self):
        return super().to_print()

### Class HuricaneEnv

In [13]:
class HuricaneEnv:
    def __init__(self):
        self.deadline = None
        self.graph = None
        self.number_of_vertices = None
        self.agents = []
        self.epoch = 0
        self.total_rescued = 0
    def read_env_from_file(self, file_name):
            file = open(file_name, "r")
            vertices = []
            edges = []
            for line in file:
                words = line.split()
                if not words: # if the list is empty
                    continue
                first_word = words[0]
                if first_word[0] == ';':
                    continue
                if first_word[1]=='N':
                    self.number_of_vertices = int(words[1])
                    continue
                if first_word[1]=='D':
                    self.deadline = float(words[1])
                    continue
                if first_word[1]=='V':
                    key = int(first_word[2:])
                    if len(words)>1:
                        second_word = words[1]
                        if second_word[0] == ';':
                            number_of_people = 0
                        else:
                            number_of_people = int(second_word[1:])
                    else:
                        number_of_people = 0
                    vertex = Vertex(key,number_of_people)
                    vertices.append(vertex)
                    continue
                if first_word[1]=='E':
                    key = int(first_word[2:])
                    second_word = words[1]
                    third_word = words[2]
                    fourth_word = words[3][1:]
                    vertex1 = Vertex.get_vertex(vertices, int(second_word))
                    vertex2 = Vertex.get_vertex(vertices, int(third_word))
                    weight = int(fourth_word)
                    edge = Edge(vertex1, vertex2, weight, key)
                    edges.append(edge)
            self.graph=Graph(vertices,edges)
            
                    
    def end(self):
        if not self.graph.more_people():
            print("All people rescued")
            return True
        for i, agent in enumerate(self.agents):
            if not agent.is_terminated:
                break
            if i==len(self.agents)-1:
                print("All agents terminated")
                return True
        if self.epoch>=np.floor(self.deadline):
            print("Deadline")
            return True
        return False
    
    def run_env(self):
        number_of_agents = int(input("How many agents?\n"))
        for i in range(number_of_agents):
            if i==0:
                print("What type of agent #{}?".format(i+1))
                print("From part I: 'h': Human, 'g1': greedy, 's': saboteur,")
                print("From part II: 'g': greedy, 'a1': A*, 'rt': RT-A* (simplified)")
                type = input()
            else:
                type = input("What type of agent #{}?\n".format(i+1))
            position = int(input("position?\n"))
            if type == 'h':
                agent = Human_Agent([self.graph], position, i+1)
            else:
                if type == 'g1':
                    agent = Greedy_Agent([self.graph], position,i+1)
                else:
                    if type == 's':
                        number_of_NO_OPS = int(input("number of NO_OPS?\n"))
                        agent = Saboteur_Agent([self.graph], position, number_of_NO_OPS,i+1)
                    else:
                        if type == 'g':
                            agent = Greedy_Agent_2([self.graph], position,i+1)
                        else:
                            if type == 'a1':
                                agent = A_star_Agent([self.graph], position,i+1)
                            else:
                                if type == 'rt':
                                    L = int(input("What is your limit (L)? (Recomanded: 10)\n"))
                                    agent = Simplified_RT_A_star_agent([self.graph], position,i+1, L)
                                else:
                                    print("Invalid type")
            self.agents.append(agent)

        while (not self.end()):
            for agent in self.agents:
                action = agent.get_action()
                if action.action_type == TRAVERSE:
                    agent.agent_move(action.argument)
                    continue
                if action.action_type == TERMINATE:
                    agent.agent_terminate()
                    continue
                if action.action_type == NO_OP:
                    continue
                if action.action_type == BLOCK:
                    agent.agent_block(action.argument)
            self.epoch += 1
            self.calc_score()
            print(self.to_print())
        
    def calc_score(self):
        self.total_rescued = 0
        for agent in self.agents:
            self.total_rescued += agent.score
        return 

    
    def to_print(self, include_graph = True):
        pr="ENV STATE: EPOCH {}___________________________\n".format(str(self.epoch))
        pr+="\tdeadline:\t"+str(self.deadline)+"\n"
        if include_graph:
            pr+="\tgraph:\n"
            graph_str = self.graph.to_print()
            lines = graph_str.splitlines()
            for i, line in enumerate(lines):
                pr+="\t\t"+line+"\n"
        pr+="\tTotal number people rescued: {}\n".format(self.total_rescued)
        pr+="\tAgents:\n"
        for i, agent in enumerate(self.agents):
            agent_str = agent.to_print()
            lines = agent_str.splitlines()
            pr+="\t\t Agent #{}\n".format(i+1)
            for i, line in enumerate(lines):
                pr+="\t\t\t"+line+"\n"
        pr+="__________________________________________________"
        return pr

## Part II

### heuristic

In [14]:
def heu1(graph, position_key):
    position = Position(graph, position_key)
    for edge in graph.edges:
        edge.weight = 1
    if position.start.number_of_people != 0:
        return 0
    
    shortest_length = float("inf")
    for vertex in graph.vertices:
        if position.start == vertex:
            continue
        if vertex.number_of_people > 0:
            path = graph.Dijkstra(position.start, vertex)
            if path==None: continue
            if len(path)<shortest_length:
                shortest_length = len(path)
    if shortest_length == float("inf"):
        return 0
    return shortest_length-1

### class Greedy_Agent_2

In [15]:
import copy
class Greedy_Agent_2(Agent):
    def __init__(self, observations, position,key):
        super().__init__(observations, position,key)
        
    def get_action(self):
        if not self.position.in_vertex():
            return Action(TRAVERSE)
        action = Action(TERMINATE)
        min_heu = float('inf')
        for vertex in self.graph.neighbours_of(self.position.start):
        #for vertex in self.graph.vertices:
            #if Vertex.is_same_vertex(vertex, self.position.start): continue
            heu_cost = heu1(copy.deepcopy(self.graph), vertex.key)
            step_cost = self.graph.find_distance(self.position.start, vertex)
            if heu_cost+step_cost<min_heu:
                min_heu = heu_cost+step_cost
                action = Action(TRAVERSE, vertex)
        return action
    
    def to_print(self):
        return super().to_print()

### class Node

In [16]:
import copy
class Node():
    def __init__(self, graph, position, g_cost, parent):
        self.graph = graph
        self.position = position
        self.g_cost = g_cost
        self.h_cost = heu1(copy.deepcopy(self.graph), position.start.key)
        self.parent = parent
        
    
    def expand(self):
        counter = 0
        children = []
        neighbours = self.graph.neighbours_of(self.position.start)
        for vertex in neighbours:
            counter+=1
            g_cost = self.graph.find_distance(self.position.start,vertex)
            new_graph = copy.deepcopy(self.graph)
            new_vertex = new_graph.get_vertex(vertex.key)
            new_vertex.number_of_people = 0
            new_position = Position(self.graph, new_vertex.key)
            new_node = Node(new_graph, new_position,g_cost, self)
            children.append(new_node)
        return children
    
    def f_cost(self):
        return self.g_cost + self.h_cost
    
    def goal_test(self):
        return not self.graph.more_people()

### class A_star_agent

In [17]:

FAILURE = -1
class A_star_Agent(Agent):
    def __init__(self, observations, position,key):
        super().__init__(observations, position,key)
        self.open = [Node(self.graph, self.position,0, None)]
        self.closed = []
        self.num_of_expands = 0
        self.in_calculation = False
        self.time_past = 0
        
    def get_action(self):
        node = self.A_star_alg()
        if node == FAILURE:
            self.in_calculation = False
            return Action(TERMINATE)
        if node == NO_OP:
            return Action(NO_OP)
        if node.parent == None:
            self.in_calculation = False
            #TODO: What to do if we are on sucess? Terminate or no_op?
            return Action(TERMINATE)
        parent = node.parent
        grand_parent = parent.parent
        while grand_parent !=None:
            node = parent
            parent = node.parent
            grand_parent = parent.parent
        self.in_calculation = False
        self.num_of_expands = 0
        return Action(TRAVERSE, node.position.start)
        
        
    def A_star_alg(self):
        if not self.in_calculation:
            self.open = [Node(self.graph, self.position,0, None)]
            self.closed = []
        self.in_calculation = True
        while True:
            if self.num_of_expands >= LIMIT:
                print("LIMIT")
                return FAILURE
            if not self.open:
                return FAILURE
            node = self.open[0]
            if self.goal_test(node):
                return node
            if self.time_past==1:
                self.time_past = 0
                return NO_OP
            self.open = self.open[1:]
            need_to_insert = True
            for node_tag in self.closed:
                if self.same_state(node, node_tag):
                    if Node.f_cost(node_tag)<Node.f_cost(node):
                        need_to_insert = False
                        break
            if need_to_insert:
                self.closed.append(node)
                self.insert_all(node.expand())
    
    def insert_all(self, nodes):
        self.num_of_expands += 1
        self.time_past+=T
        self.open = self.open + nodes
        self.open.sort(key = Node.f_cost)
    
    def same_state(self, node1, node2):
        if not Graph.is_same_graph(node1.graph, node2.graph): 
            return False
        if not Position.is_same_position(node1.position, node2.position):
            return False
        return True
    
    def goal_test(self, node):
        return node.goal_test()
        
    def to_print(self):
        return super().to_print()

### class Simplified_RT_A_star_agent

In [18]:
class Simplified_RT_A_star_agent(Agent):
    def __init__(self, observations, position,key, L):
        super().__init__(observations, position,key)
        self.open = [Node(self.graph, self.position,0, None)]
        self.closed = []
        self.num_of_expands = 0
        self.L = L
        self.time_past = 0
        self.num_of_expands_in_epoch = 0 
        
    def get_action(self):
        node = self.A_star_alg()
        if node == FAILURE:
            self.in_calculation = False
            return Action(TERMINATE)
        if node == NO_OP:
            return Action(NO_OP)
        if node.parent == None:
            self.in_calculation = False
            #TODO: What to do if we are on sucess? Terminate or no_op?
            return Action(TERMINATE)
            
        parent = node.parent
        grand_parent = parent.parent
        while grand_parent !=None:
            node = parent
            parent = node.parent
            grand_parent = parent.parent
        self.in_calculation = False
        self.num_of_expands_in_epoch = 0 
        return Action(TRAVERSE, node.position.start)
        
        
    def A_star_alg(self):
        if not self.in_calculation:
            self.open = [Node(self.graph, self.position,0, None)]
            self.closed = []
        self.in_calculation = True
        self.num_of_expands_in_epoch = 0 
        while True:
           #if self.num_of_expands_in_epoch >= LIMIT:
                #print("LIMIT")
                #return FAILURE
            if not self.open:
                return FAILURE               
            node = self.open[0]
            if self.goal_test(node) or self.num_of_expands_in_epoch >= self.L:
                return node
            
            if self.time_past==1:
                self.time_past = 0
                return NO_OP
            self.open = self.open[1:]
            if self.num_of_expands_in_epoch >= self.L:
                return node
            
            need_to_insert = True
            for node_tag in self.closed:
                if self.same_state(node, node_tag):
                    if Node.f_cost(node_tag)<Node.f_cost(node):
                        need_to_insert = False
                        break
            if need_to_insert:
                self.closed.append(node)
                self.insert_all(node.expand())
    
    def insert_all(self, nodes):
        self.num_of_expands += 1
        self.num_of_expands_in_epoch+=1
        self.time_past+=T
        self.open = self.open + nodes
        self.open.sort(key = Node.f_cost)
    
    def same_state(self, node1, node2):
        if not Graph.is_same_graph(node1.graph, node2.graph): 
            return False
        if not Position.is_same_position(node1.position, node2.position):
            return False
        return True
    
    def goal_test(self, node):
        return node.goal_test()
        
    def to_print(self):
        return super().to_print()+"\n(L)imit: {}".format(self.L)

### Run the program

In [22]:
def set_trace(): pass
#This is what I submitted
import sys
input_from_file = True
#output_file = open("output.txt", "w")
if input_from_file:
    #def print(message):
        #sys.stdout.write(message)
        #output_file.write(message)
    if False:
        def input(message=""):
            print(message)
            p = next(inp_file)
            print(p)
            if p[-1] == '\n':
                p = p[0:-1]
            return p
    #open('output.txt', 'w').close()
    #output = open('output.txt', 'w')
    for T in [0]:
        print("--------------------------\n--------------------------\n")
        print("\nNEW RUNNING: configuration, T = "+str(T)+"\n")
        inp_file = open("inp.txt")
        inp_file_iter = iter(inp_file)

        env=HuricaneEnv()
        env.read_env_from_file("input2.txt")
        env.run_env()
        inp_file.close()
    #output_file.close()

--------------------------
--------------------------


NEW RUNNING: configuration, T = 0

How many agents?
1
What type of agent #1?
From part I: 'h': Human, 'g1': greedy, 's': saboteur,
From part II: 'g': greedy, 'a1': A*, 'rt': RT-A* (simplified)
a1
position?
4
ENV STATE: EPOCH 1___________________________
	deadline:	20.0
	graph:
		Vertices:
			1, (0 people)
			2, (0 people)
			3, (0 people)
			4, (0 people)
			5, (5 people)
			6, (0 people)
			7, (0 people)
		Edges:
			W1: (1, 3) weight 2
			W2: (1, 2) weight 3
			W3: (2, 3) weight 4
			W4: (1, 6) weight 1
			W5: (1, 7) weight 5
			W6: (3, 6) weight 1
			W7: (3, 4) weight 7
			W8: (4, 5) weight 4
			W9: (4, 7) weight 10
	Total number people rescued: 4
	Agents:
		 Agent #1
			A_star_Agent
			actions_counter: 1
			score: 4
			is_terminated: False
			position:
				From 4 to 5. Distance to destination: 3
			In a calculation: False
__________________________________________________
ENV STATE: EPOCH 2___________________________
	deadline: